In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
# 데이터 경로 설정
data_dir = "C://temp//cat_plant_dataset"  # 여기에 실제 데이터 경로를 입력하세요
# 이미지 크기 및 배치 크기 설정
img_height, img_width = 150, 150
batch_size = 32
# 데이터 제너레이터 생성
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2  # 20%를 검증 데이터로 사용
)
# 학습 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
)
# 검증 데이터 생성기
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
)
# 클래스 수 확인
num_classes = len(train_generator.class_indices)
print(f"클래스 수: {num_classes}")
print("클래스 이름:", train_generator.class_indices)
# 모델 생성
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
optimizer=optimizers.Nadam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
# 모델 컴파일
model.compile(optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy'])
# 모델 학습
epochs = 20
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)
# 학습 결과 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()
# 모델 저장
model.save('plant_classification_model.h5')
# 모델 평가
test_loss, test_acc = model.evaluate(validation_generator)
print(f'테스트 정확도: {test_acc:.2f}')